In [ ]:
import pickle
import pandas as pd
import requests


from flask import Flask, jsonify
import pandas as pd
import threading
import json  # For JSON beautification

In [ ]:


# Load the preprocessor and model
with open("preprocessor.pkl", "rb") as f:
    preprocessor = pickle.load(f)

with open("model.pkl", "rb") as f:
    model = pickle.load(f)




In [ ]:
# Load the LabelEncoders
with open("gender_encoder.pkl", "rb") as f:
    le_gender = pickle.load(f)

with open("treatment_encoder.pkl", "rb") as f:
    le_treatment = pickle.load(f)


In [ ]:
# Example of new data for prediction
# new_sample = pd.DataFrame({
#     'age_cleaned': [30],
#     'gender_encoded': [1],  # 1 for Female based on LabelEncoder
#     'complaint_cleaned': ["sadness and fear"],
#     'duration_cleaned': [5]
# })

new_sample = pd.DataFrame({
    'age_cleaned': [30],
    'gender_encoded': [le_gender.transform(['Male'])[0]],  # Automatically transform 'female' to its encoded value
    'complaint_cleaned': ["sadness and fear"],
    'duration_cleaned': [5]
})

# Preprocess the new data
new_sample_transformed = preprocessor.transform(new_sample)



# Predict using the loaded model
predicted_treatment_encoded = model.predict(new_sample_transformed)

# Convert the predicted encoded value back to the original string value
predicted_treatment = le_treatment.inverse_transform(predicted_treatment_encoded.astype(int))

print("Predicted Treatment Plan (original string):", predicted_treatment)


Predicted Treatment Plan (original string): ['RESP, 2MG, 1-0-2, 1 Months, -\n\nTHP, 2MG, 1-0-1, 1 Months, -']


In [ ]:


# Function to take input from the user
def get_user_input():
    print("Please enter the following details:")
    age = int(input("Age: "))
    gender = input("Gender (male/female): ").strip()
    complaint = input("Complaint (e.g., sadness and fear): ").strip()
    duration = int(input("Duration in years(numeric only): "))

    # Ensure gender is valid
    if gender not in ['Male', 'Female']:
        raise ValueError("Invalid gender. Please enter 'Male' or 'Female' with capital M and F.")

    # Transform gender to encoded form
    gender_encoded = le_gender.transform([gender])[0]

    # Create the input DataFrame
    new_sample = pd.DataFrame({
        'age_cleaned': [age],
        'gender_encoded': [gender_encoded],
        'complaint_cleaned': [complaint],
        'duration_cleaned': [duration]
    })

    return new_sample



In [ ]:

# Main function to predict treatment
def predict_treatment():
    try:
        # Get user input
        user_input = get_user_input()

        # Preprocess the new data
        user_input_transformed = preprocessor.transform(user_input)

        # Predict using the loaded model
        predicted_treatment_encoded = model.predict(user_input_transformed)

        # Convert the predicted encoded value back to the original string value
        predicted_treatment = le_treatment.inverse_transform(predicted_treatment_encoded.astype(int))

        # Output the result
        print("\nPredicted Treatment Plan (original string):", predicted_treatment[0])
        # print("Input Gender:", le_gender.inverse_transform([user_input['gender_encoded'][0]])[0])

    except ValueError as e:
        print("Error:", e)

# Run the prediction function
if __name__ == "__main__":
    predict_treatment()

Please enter the following details:
Age: 32
Gender (male/female): Female
Complaint (e.g., sadness and fear): SADNESS,FEAR/PALPITION, SUSPECIAUSNESS, HEARINING OF VOICES
Duration in months years(numeric only): 6

Predicted Treatment Plan (original string): RISPERIDONE, 4mg, 1-0-1, 1 Months, -

TRIHEXYPHENIDYL, 2mg, 1-1-0, 1 Months, -

OLANZAPINE, 10mg, 0-0-1, 1 Months, -

LORAZEPAM, 2MG, 1-0-1, 1 Months, -


In [ ]:


# Initialize Flask app
app = Flask(__name__)

# Define the /getdata endpoint
@app.route('/data', methods=['GET'])
def fetch_data():
    # Load the data and analyze it
    df = pd.read_csv('/content/afzalpur_encoded.csv')
    complaints_json = {'complaints': df['complaint_cleaned'].tolist()}

  #   # Pretty-print the JSON
  #   formatted_json = json.dumps(complaints_json, indent=4)

  # # Update the file path if needed
  #   result = formatted_json

    # # Beautify and return the JSON response
    # pretty_json = json.dumps(result, indent=4)
    # return app.response_class(pretty_json, mimetype='application/json')

    # Return the JSON response directly
    return jsonify(complaints_json)


# Function to run Flask in a thread
def run_flask():
    app.run(host='0.0.0.0', port=5121)


# Start Flask server in a new thread
thread = threading.Thread(target=run_flask)
thread.start()



 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5121
 * Running on http://172.28.0.12:5121
INFO:werkzeug:Press CTRL+C to quit


In [ ]:
# Test the API from the same Colab notebook
try:
    response = requests.get('http://127.0.0.1:5121/data')
    print("Beautified JSON Response:")
    print(json.dumps(response.json(), indent=4))  # Beautify the response
except Exception as e:
    print(f"Error: {e}")

INFO:werkzeug:127.0.0.1 - - [18/Nov/2024 21:33:52] "GET /data HTTP/1.1" 200 -


Beautified JSON Response:
{
    "complaints": [
        "sadness,fear/palpition, suspeciausness, hearining of voices",
        "seizure",
        "lack of sleep, self care poor, hallucination",
        "muttering to self, irritability, suspicious",
        "headache, giddiness, fear, palpition",
        "delay development, gtcs",
        "lack of sleep, talking self, suspiciousness",
        "lack of sleep,abnormal behaviour , violent behaviour",
        "k/c/o f20",
        "idd with behaviour problem",
        "muttering to self , aggresive voilent behavioral",
        "hearing voices not heared by others",
        "excessive talking , over religiosity , talking to self",
        "talking to self smiling to self suspiciouness",
        "episodes of involuntary movements of ul and il",
        "k/c/o",
        "episodes of excessive talking , irritabilty , increased energy",
        "episodes of excessive talking , irritabilty , increased energy",
        "decreased social interaction

In [ ]:
response = requests.get('http://127.0.0.1:5122/data')
print(response.text)  # Check the raw response content


INFO:werkzeug:127.0.0.1 - - [18/Nov/2024 21:32:24] "GET /data HTTP/1.1" 200 -


"{\n    \"complaint_cleaned\": [\n        \"sadness,fear/palpition, suspeciausness, hearining of voices\",\n        \"seizure\",\n        \"lack of sleep, self care poor, hallucination\",\n        \"muttering to self, irritability, suspicious\",\n        \"headache, giddiness, fear, palpition\",\n        \"delay development, gtcs\",\n        \"lack of sleep, talking self, suspiciousness\",\n        \"lack of sleep,abnormal behaviour , violent behaviour\",\n        \"k/c/o f20\",\n        \"idd with behaviour problem\",\n        \"muttering to self , aggresive voilent behavioral\",\n        \"hearing voices not heared by others\",\n        \"excessive talking , over religiosity , talking to self\",\n        \"talking to self smiling to self suspiciouness\",\n        \"episodes of involuntary movements of ul and il\",\n        \"k/c/o\",\n        \"episodes of excessive talking , irritabilty , increased energy\",\n        \"episodes of excessive talking , irritabilty , increased energy\"

In [ ]:


# Function to fetch complaints from the API
def fetch_complaints():
    try:
        response = requests.get('http://127.0.0.1:5121/data')  # API URL
        complaints_data = response.json()
        complaints_list = complaints_data['complaints']
        return complaints_list
    except Exception as e:
        print(f"Error fetching complaints: {e}")
        return []

# Function to take user input
def get_user_input():
    print("Fetching complaints from the API...")
    complaints_list = fetch_complaints()

    if not complaints_list:
        print("No complaints data available. Please check the API.")
        return None

    print("\nAvailable Complaints:")
    print("\n".join(complaints_list))
    print("\nCopy and paste the exact or relevant complaint combination from the above list:")

    # Collect user inputs
    age = int(input("Age: "))
    gender = input("Gender (male/female): ").strip().lower()
    complaint = input("Complaint: ").strip()  # User pastes the complaint here
    duration = int(input("Duration in months (numeric only): "))

    # Ensure gender is valid
    if gender not in ['male', 'female']:
        raise ValueError("Invalid gender. Please enter 'male' or 'female'.")

    # Transform gender to encoded form
    gender_encoded = le_gender.transform([gender])[0]

    # Create the input DataFrame
    new_sample = pd.DataFrame({
        'age_cleaned': [age],
        'gender_encoded': [gender_encoded],
        'complaint_cleaned': [complaint],
        'duration_cleaned': [duration]
    })

    return new_sample

# Main function to predict treatment
def predict_treatment():
    try:
        # Get user input
        user_input = get_user_input()

        if user_input is None:
            return

        # Preprocess the new data
        user_input_transformed = preprocessor.transform(user_input)

        # Predict using the loaded model
        predicted_treatment_encoded = model.predict(user_input_transformed)

        # Convert the predicted encoded value back to the original string value
        predicted_treatment = le_treatment.inverse_transform(predicted_treatment_encoded.astype(int))

        # Output the result
        print("\nPredicted Treatment Plan (original string):", predicted_treatment[0])
        print("Input Gender:", le_gender.inverse_transform([user_input['gender_encoded'][0]])[0])

    except ValueError as e:
        print("Error:", e)

# Run the prediction function
if __name__ == "__main__":
    predict_treatment()


INFO:werkzeug:127.0.0.1 - - [18/Nov/2024 21:34:29] "GET /data HTTP/1.1" 200 -


Fetching complaints from the API...

Available Complaints:
sadness,fear/palpition, suspeciausness, hearining of voices
seizure
lack of sleep, self care poor, hallucination
muttering to self, irritability, suspicious
headache, giddiness, fear, palpition
delay development, gtcs
lack of sleep, talking self, suspiciousness
lack of sleep,abnormal behaviour , violent behaviour
k/c/o f20
idd with behaviour problem
muttering to self , aggresive voilent behavioral
hearing voices not heared by others
excessive talking , over religiosity , talking to self
talking to self smiling to self suspiciouness
episodes of involuntary movements of ul and il
k/c/o
episodes of excessive talking , irritabilty , increased energy
episodes of excessive talking , irritabilty , increased energy
decreased social interaction , suspiciousness
talking to self smiling to self
agressive behavior, irrilavant talk, smiling to self
abnormal behavior, distrubed sleep, irrelevant talk
abnormal behavior, distrubed sleep, suspi

In [ ]:


# Function to fetch complaints from the API
def fetch_complaints():
    try:
        response = requests.get('http://127.0.0.1:5121/data')  # API URL
        complaints_data = response.json()
        complaints_list = complaints_data['complaints']
        return complaints_list
    except Exception as e:
        print(f"Error fetching complaints: {e}")
        return []

# Function to filter complaints based on user input
def filter_complaints(user_complaint, complaints_list):
    filtered_complaints = [complaint for complaint in complaints_list if user_complaint.lower() in complaint.lower()]
    return filtered_complaints

# Function to take user input
def get_user_input():
    # Ask user for their primary complaint
    user_complaint = input("Enter your primary complaint (e.g., headache, dizziness): ").strip().lower()

    # Fetch complaints from the API
    print("Fetching complaints from the API...")
    complaints_list = fetch_complaints()

    if not complaints_list:
        print("No complaints data available. Please check the API.")
        return None

    # Filter complaints based on user input
    filtered_complaints = filter_complaints(user_complaint, complaints_list)

    if not filtered_complaints:
        print("No relevant complaints found for your input.")
        return None

    print("\nRelevant Complaints:")
    print("\n".join(filtered_complaints))
    print("\nCopy and paste the exact or relevant complaint combination from the above list when complaint is asked next:")

    # Collect additional user inputs
    age = int(input("Age: "))
    gender = input("Gender (Male/Female): ").strip()
    complaint = input("Complaint: ").strip()  # User pastes the exact or relevant complaint here
    duration = int(input("Duration in years (numeric only): "))

    # Ensure gender is valid
    if gender not in ['Male', 'Female']:
        raise ValueError("Invalid gender. Please enter 'Male' or 'Female' with capital M & F.")

    # Transform gender to encoded form
    gender_encoded = le_gender.transform([gender])[0]

    # Create the input DataFrame
    new_sample = pd.DataFrame({
        'age_cleaned': [age],
        'gender_encoded': [gender_encoded],
        'complaint_cleaned': [complaint],
        'duration_cleaned': [duration]
    })

    return new_sample

# Main function to predict treatment
def predict_treatment():
    try:
        # Get user input
        user_input = get_user_input()

        if user_input is None:
            return

        # Preprocess the new data
        user_input_transformed = preprocessor.transform(user_input)

        # Predict using the loaded model
        predicted_treatment_encoded = model.predict(user_input_transformed)

        # Convert the predicted encoded value back to the original string value
        predicted_treatment = le_treatment.inverse_transform(predicted_treatment_encoded.astype(int))

        # Output the result
        print("\nPredicted Treatment Plan (original string):", predicted_treatment[0])
        print("Input Gender:", le_gender.inverse_transform([user_input['gender_encoded'][0]])[0])

    except ValueError as e:
        print("Error:", e)

# Run the prediction function
if __name__ == "__main__":
    predict_treatment()


Enter your primary complaint (e.g., headache, dizziness): sadness


INFO:werkzeug:127.0.0.1 - - [18/Nov/2024 21:41:18] "GET /data HTTP/1.1" 200 -


Fetching complaints from the API...

Relevant Complaints:
sadness,fear/palpition, suspeciausness, hearining of voices
lack of interest, sadness, disturbed sleep,
lack of interest, disturbed sleep, sadness
lack of interest, disturbed sleep, sadness
choric headache, palpitation, chest pain, sadness, lack of sleep

Copy and paste the exact or relevant complaint combination from the above list when complaint is asked next:
Age: 31
Gender (Male/Female): Male
Complaint: choric headache, palpitation, chest pain, sadness, lack of sleep
Duration in years (numeric only): 1

Predicted Treatment Plan (original string): RISPERIDONE, 4mg, 1-0-1, 1 Months, -

TRIHEXYPHENIDYL, 2mg, 1-0-1, 1 Months, -

OLANZAPINE, 10mg, 0-0-1/2, 1 Months, -
Input Gender: Male


In [ ]:


# Fetch data from the API
def fetch_complaints():
    try:
        response = requests.get('http://127.0.0.1:5122/data')
        if response.status_code == 200:
            return response.json()  # Parsed JSON response
        else:
            print("Failed to fetch data from API. Status code:", response.status_code)
            return {}
    except Exception as e:
        print(f"Error fetching data from API: {e}")
        return {}

# Match and get complaints from the API data
def get_matching_complaints(api_data, user_complaint):
    matching_complaints = []

    complaints = gender_data.get("complaint_cleaned", [])
    if any(user_complaint.lower() in c.lower() for c in complaints):
        matching_complaints.extend(complaints)
    return list(set(matching_complaints))  # Remove duplicates

# Function to take input from the user
def get_user_input(api_data):
    print("Please enter the following details:")
    age = int(input("Age: "))
    gender = input("Gender (male/female): ").strip().lower()
    duration = int(input("Duration in months (numeric only): "))

    # Ensure gender is valid
    if gender not in ['male', 'female']:
        raise ValueError("Invalid gender. Please enter 'male' or 'female'.")

    # Get user complaint and match with API data
    user_complaint = input("Enter your complaint (e.g., sadness, headache): ").strip()
    matching_complaints = get_matching_complaints(api_data, user_complaint)

    if matching_complaints:
        print("\nMatching complaints found:")
        for complaint in matching_complaints:
            print(f"- {complaint}")
        selected_complaint = input(
            "\nCopy and paste the exact or relevant complaint combination from above: "
        ).strip()
    else:
        print("No matching complaints found. Using the entered complaint.")
        selected_complaint = user_complaint

    # Transform gender to encoded form
    gender_encoded = le_gender.transform([gender])[0]

    # Create the input DataFrame
    new_sample = pd.DataFrame({
        'age_cleaned': [age],
        'gender_encoded': [gender_encoded],
        'complaint_cleaned': [selected_complaint],
        'duration_cleaned': [duration]
    })

    return new_sample

# Main function to predict treatment
def predict_treatment():
    # Fetch complaints data from API
    api_data = fetch_top_complaints()

    if not api_data:
        print("Error: Could not retrieve complaints data from the API.")
        return

    try:
        # Get user input
        user_input = get_user_input(api_data)

        # Preprocess the new data
        user_input_transformed = preprocessor.transform(user_input)

        # Predict using the loaded model
        predicted_treatment_encoded = model.predict(user_input_transformed)

        # Convert the predicted encoded value back to the original string value
        predicted_treatment = le_treatment.inverse_transform(predicted_treatment_encoded.astype(int))

        # Output the result
        print("\nPredicted Treatment Plan (original string):", predicted_treatment[0])
        print("Input Gender:", le_gender.inverse_transform([user_input['gender_encoded'][0]])[0])

    except ValueError as e:
        print("Error:", e)

# Run the prediction function
if __name__ == "__main__":
    predict_treatment()
